In [1]:
import talib 
from talib import MA_Type
import pandas as pd
import datetime
import pandas as pd
import os
from urllib.request import urlopen
import json
import os 
import numpy as np
import tushare as ts
import datetime
from datetime import timedelta
import pyecharts.options as opts
import seaborn as sns
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [9]:
all_data=pd.read_csv(r"D:\量化\每日量化选股\MACD背离选法\机器学习筛选02.csv.",encoding='GBK')
all_data

,Date,ts_code,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,pct_chg_0,signal
0,2016-11-21,000001.SZ,1,0.65,4.16,2.35,-0.48,-0.90,57.708698,0.07,-0.42,1.1660,-0.66,0.32,9.24,0.5811,1.1172,1.43,7.2561,6.9451,0.8899,1.6499,1.4830,1.2500,1.3800,1.717041e+06,1.463118e+06,761028.1784,1.586546e+07,1.351921e+07,0.842385,-0.121575,-0.073443,0.346562,0.306969,-0.686620,0.927544,1,1
1,2019-02-14,000006.SZ,1,1.08,0.16,8.13,6.38,16.11,65.676599,1.25,-1.26,3.8168,0.32,2.11,5.64,1.2510,1.9560,1.40,9.4509,8.6014,1.2264,2.5728,3.0420,3.2092,3.2092,1.349995e+05,1.348308e+05,86230.9824,7.613972e+05,7.604457e+05,0.841821,-0.189062,-0.274154,0.269576,0.305851,-0.889919,0.083333,1,1
2,2018-04-12,000009.SZ,1,0.48,-2.63,-6.72,-2.43,-5.45,53.394291,0.47,-0.68,5.3980,-5.26,-5.59,6.26,0.8175,0.9936,0.67,57.6514,66.1183,2.8934,2.0985,1.8962,0.3195,0.3195,2.149345e+05,2.120089e+05,174428.7793,1.345490e+06,1.327176e+06,0.241450,-0.124004,-0.099795,0.398470,0.451688,-0.880894,0.478463,0,1
3,2015-03-06,000011.SZ,1,2.94,2.16,11.58,23.06,52.31,61.882218,0.47,-2.49,3.7933,2.21,1.90,9.80,10.3346,12.5169,2.84,19.4143,21.8173,2.9537,3.6070,6.1477,2.5510,2.5510,5.959791e+04,1.758626e+04,14520.1889,5.840595e+05,1.723454e+05,0.844625,-0.031979,-0.405080,0.184916,0.174682,-0.950094,0.375898,1,1
4,2014-07-28,000012.SZ,1,1.27,3.34,3.74,3.34,7.34,65.427010,0.29,-0.79,2.2295,1.22,-2.14,7.19,3.3688,3.9691,2.37,9.7151,9.6187,1.9745,1.9294,1.9229,2.0862,4.1725,2.075336e+05,1.302467e+05,110546.7779,1.492166e+06,9.364740e+05,0.845346,-0.017787,-0.319315,0.260623,0.274664,0.000000,0.261771,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,2019-08-22,603997.SH,1,4.80,3.44,0.16,12.66,10.31,62.344409,0.16,-0.54,2.6451,-1.23,-6.30,7.86,0.8171,3.3844,2.85,16.6205,18.4457,2.7857,2.3361,2.4033,3.5529,3.9695,6.394132e+04,6.318708e+04,15254.9353,5.025788e+05,4.966505e+05,0.843949,-0.065119,-0.134027,0.285007,0.069086,-0.782182,0.224006,1,1
1661,2019-12-26,603997.SH,1,3.69,-3.43,2.26,-16.91,-9.04,62.100018,0.70,-0.15,1.9982,-0.58,-0.06,8.42,0.5254,2.1710,1.32,28.5025,33.1161,1.8006,4.0062,4.0528,2.0718,2.3147,1.023603e+05,6.323533e+04,15303.1803,8.618737e+05,5.324415e+05,0.843949,-0.178855,-0.030970,0.383310,0.085834,-0.988941,0.246336,1,1
1662,2021-04-01,603998.SH,1,0.38,0.76,9.36,17.53,19.85,49.476538,0.05,-0.19,2.7518,-3.11,-5.95,5.25,0.2742,0.5174,0.68,28.5703,32.7371,1.9610,2.0613,1.8801,2.2975,2.2975,4.294297e+04,4.294297e+04,22761.5945,2.254506e+05,2.254506e+05,0.079637,-0.019093,-0.113690,0.426075,0.501659,-0.658975,0.730591,1,1
1663,2021-03-30,605118.SH,1,2.01,9.36,14.58,-0.61,-4.20,53.980823,0.05,-0.54,4.8783,-4.47,-5.92,14.74,2.9443,2.9443,0.72,32.9829,36.4366,5.4980,12.8405,13.5354,0.0000,0.0000,4.055000e+04,4.100000e+03,4100.0000,5.977070e+05,6.043400e+04,0.068783,-0.190436,-0.086131,0.292553,0.227053,0.000000,0.202899,1,1


In [10]:
def report(output_df,up_chg=0,down_chg=0):
    sum_data = len(output_df)
    sum_up = len(output_df[(output_df['trade_type']==1)])
    sum_down = len(output_df[(output_df['trade_type']==0)])
    
    buy_up_5 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_5']>up_chg))])
    buy_up_10 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_10']>up_chg))])
    buy_up_20 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_20']>up_chg))])
    buy_up_30 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_30']>up_chg))])
    
    sell_dwon_5 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_5']<down_chg))])
    sell_dwon_10 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_10']<down_chg))])
    sell_dwon_20 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_20']<down_chg))])
    sell_dwon_30 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_30']<down_chg))])
                    
    print('总数据量:{}'.format(sum_data))
    print('给出做多信号数据量:{}'.format(sum_up))      
    print('给出做空信号数据量:{}'.format(sum_down))
    
    print('在判断为上涨信号时，5天后为上涨的概率:{}'.format(buy_up_5/sum_up))
    print('在判断为上涨信号时，10天后为上涨的概率:{}'.format(buy_up_10/sum_up))
    print('在判断为上涨信号时，20天后为上涨的概率:{}'.format(buy_up_20/sum_up))
    print('在判断为上涨信号时，30天后为上涨的概率:{}'.format(buy_up_30/sum_up))
    print('\n')

In [11]:
up_chg=0
down_chg=0
report(all_data,up_chg,down_chg)

总数据量:1665
给出做多信号数据量:1665
给出做空信号数据量:0
在判断为上涨信号时，5天后为上涨的概率:0.6576576576576577
在判断为上涨信号时，10天后为上涨的概率:0.8444444444444444
在判断为上涨信号时，20天后为上涨的概率:0.7015015015015015
在判断为上涨信号时，30天后为上涨的概率:0.6720720720720721




In [12]:
n = -5
neg_data = all_data[(all_data['pct_chg_5']<n) | (all_data['pct_chg_10']<n) | (all_data['pct_chg_20']<n) | (all_data['pct_chg_30']<n)]
neg_data.tail(50)

,Date,ts_code,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,pct_chg_0,signal
1464,2021-01-12,603019.SH,1,0.56,-1.58,-9.26,-18.11,-15.38,59.882617,0.57,-0.20,4.4448,-2.24,-4.83,35.80,2.0829,2.7736,0.72,87.4886,77.7502,4.7331,5.4518,5.9677,0.2507,0.2507,145072.8974,130205.0903,97779.5743,5.193610e+06,4.661342e+06,0.395768,-0.827827,-0.041760,0.391582,0.397606,-1.068278,0.373031,0,1
1469,2017-06-13,603030.SH,1,5.21,1.03,5.57,-4.71,-16.71,61.094284,1.50,-0.33,4.4232,-1.50,-0.23,30.10,4.8649,4.9149,1.24,53.8348,48.2118,3.6484,1.6201,1.5299,0.1776,0.0000,17955.3111,6435.5320,6370.0220,5.404549e+05,1.937095e+05,0.837374,-0.240094,-0.013695,0.139001,0.049300,-1.071731,0.130580,1,1
1478,2019-12-31,603090.SH,1,0.61,-0.54,-1.14,-7.36,-2.57,63.149954,0.38,-2.22,2.1974,0.60,-0.93,16.38,1.3638,3.3524,1.82,50.2085,49.8667,3.3019,3.9456,3.3735,0.6105,0.6105,10000.0000,10000.0000,4068.1300,1.638000e+05,1.638000e+05,0.833181,-0.205154,-0.425263,0.238447,0.415985,-1.163359,0.267030,0,1
1480,2020-01-09,603096.SH,1,3.26,2.41,-2.59,-8.08,-12.00,60.494549,0.04,-0.43,1.7024,0.58,-1.05,58.95,1.3213,1.5662,0.96,33.1227,31.4097,4.2467,8.6134,8.4574,1.0178,1.0178,13530.8000,5430.8000,4581.8500,7.976407e+05,3.201457e+05,0.848884,-0.392967,-0.048768,0.218496,0.107677,-1.006440,0.200399,0,1
1481,2018-05-10,603098.SH,1,0.31,0.78,4.08,-1.38,-7.81,55.418975,0.06,-0.36,1.8884,-4.20,-2.95,16.44,2.1108,2.1108,1.41,32.8141,31.4851,3.8315,3.0782,2.8052,1.8248,1.8248,40001.0000,6251.0000,6251.0000,6.576164e+05,1.027664e+05,0.845974,-0.138093,-0.436681,0.366448,0.548803,-1.137114,0.848987,1,1
1482,2020-04-20,603098.SH,1,2.86,3.00,3.00,-5.41,-5.32,63.484296,1.06,-1.31,3.9786,-0.58,-4.40,9.70,0.5194,2.5427,1.83,21.1544,18.1825,2.3224,1.5885,1.3088,1.0309,1.0309,48001.2000,48001.2000,9805.2000,4.656116e+05,4.656116e+05,0.845974,-0.096713,-0.582960,0.173383,0.199816,-0.955064,0.120626,1,1
1487,2018-05-04,603116.SH,1,0.76,1.01,5.20,3.10,-5.26,55.492645,0.44,-0.74,1.4700,-2.05,-2.20,14.57,0.9132,1.0739,0.93,15.9321,15.7848,1.6998,1.8730,1.8555,1.6813,1.6813,41720.0000,18070.5710,15366.4885,6.078604e+05,2.632882e+05,0.242275,-0.191396,-0.400892,0.218947,0.306094,-1.091230,0.199446,1,1
1488,2018-05-10,603117.SH,1,3.42,-0.45,1.46,0.45,-20.11,61.434953,0.37,-0.45,4.0747,-0.22,-3.27,8.78,2.3811,2.6148,1.75,38.9587,35.4837,1.7547,6.0102,5.2723,1.1390,1.1390,46232.0932,28174.3831,25655.9025,4.059178e+05,2.473711e+05,0.853960,-0.223879,-0.379424,0.149222,0.073971,-1.192987,0.411026,1,1
1491,2018-03-13,603128.SH,1,0.59,-7.00,-13.59,-5.91,-15.83,58.748566,0.05,-3.22,5.3572,0.46,-6.09,8.47,2.1716,4.0139,1.55,38.0757,38.1112,2.3354,1.1652,1.0090,1.0560,1.0560,100541.5153,91108.1711,49292.2892,8.515866e+05,7.716862e+05,0.844272,-0.319377,-0.127197,0.207885,0.320142,-0.707968,0.206143,0,1
1495,2019-09-24,603159.SH,1,7.94,6.33,0.47,-13.34,-16.64,65.736733,1.02,-1.90,5.3942,0.47,-5.68,13.33,4.2624,11.4921,2.19,43.4749,72.9084,4.5188,2.9281,3.2613,0.8038,0.8038,14000.0000,14000.0000,5192.6000,1.866200e+05,1.866200e+05,0.844072,-0.194154,-0.101887,0.080940,0.014555,-0.977785,0.007705,1,1


In [13]:
print('止损比例：{}'.format(len(neg_data)/len(all_data)))

止损比例：0.21861861861861862


In [14]:
for n in range(-10,0,1):
    print(n)
    neg_data = all_data[(all_data['pct_chg_5']<n) | (all_data['pct_chg_10']<n) | (all_data['pct_chg_20']<n) | (all_data['pct_chg_30']<n)]
    print('止损比例：{}'.format(len(neg_data)/len(all_data)))

-10
止损比例：0.0996996996996997
-9
止损比例：0.11351351351351352
-8
止损比例：0.13513513513513514
-7
止损比例：0.16096096096096096
-6
止损比例：0.1837837837837838
-5
止损比例：0.21861861861861862
-4
止损比例：0.2618618618618619
-3
止损比例：0.32012012012012014
-2
止损比例：0.384984984984985
-1
止损比例：0.46126126126126127


In [15]:
for n in range(0,31,1):
    print(n)
    neg_data = all_data[(all_data['pct_chg_5']>n) | (all_data['pct_chg_10']>n) | (all_data['pct_chg_20']>n) | (all_data['pct_chg_30']>n)]
    print('止盈比例：{}'.format(len(neg_data)/len(all_data)))

0
止盈比例：0.9363363363363363
1
止盈比例：0.8912912912912913
2
止盈比例：0.8408408408408409
3
止盈比例：0.7837837837837838
4
止盈比例：0.7207207207207207
5
止盈比例：0.6660660660660661
6
止盈比例：0.6216216216216216
7
止盈比例：0.5723723723723724
8
止盈比例：0.5225225225225225
9
止盈比例：0.4774774774774775
10
止盈比例：0.43843843843843844
11
止盈比例：0.4102102102102102
12
止盈比例：0.3825825825825826
13
止盈比例：0.35195195195195195
14
止盈比例：0.33153153153153153
15
止盈比例：0.3003003003003003
16
止盈比例：0.2768768768768769
17
止盈比例：0.25405405405405407
18
止盈比例：0.23723723723723725
19
止盈比例：0.2228228228228228
20
止盈比例：0.1993993993993994
21
止盈比例：0.17957957957957957
22
止盈比例：0.16996996996996996
23
止盈比例：0.15735735735735737
24
止盈比例：0.14654654654654656
25
止盈比例：0.13633633633633635
26
止盈比例：0.12372372372372373
27
止盈比例：0.11291291291291292
28
止盈比例：0.1057057057057057
29
止盈比例：0.09309309309309309
30
止盈比例：0.08468468468468468


In [16]:
def pct_mean(all_up_data):
    print('5days',all_up_data['pct_chg_5'].mean())
    print('10days',all_up_data['pct_chg_10'].mean())
    print('20days',all_up_data['pct_chg_20'].mean())
    print('30days',all_up_data['pct_chg_30'].mean())

In [17]:
pct_mean(all_data)

5days 2.2487267267267272
10days 4.6418078078078
20days 6.274330330330323
30days 8.392996996996976
